# 0 Import Packages

In [1]:
from pathlib import Path
import sys
import os
import requests
import pandas as pd
import json
from tqdm import tqdm


# 1 Metadata

## 1.1 Paths

In [2]:
ROOT_DIR = os.path.join(Path.cwd().parents[1], "config")

## 1.2 Custom Packages

In [3]:
sys.path.append(str(ROOT_DIR))

import keys
import config

# 2 Scrap Data

## 2.1 Get EIC Listing

In [4]:
url = "https://agsi.gie.eu/api/about"

headers = {}
headers["x-key"] = keys.AGSI


resp = requests.get(url, headers=headers)
data = resp.json()

In [5]:
EIC_listing_file = os.path.join(config.DATA_AGSI_DIR, "EIC_listing.json")

In [6]:
with open(EIC_listing_file, 'w') as file:
    json.dump(data, file)

## 2.2 Extract EIC Listing

In [7]:
with open(EIC_listing_file,"r") as file:
    EIC_listing_data = json.load(file)

In [8]:
country_name_list = list(data["SSO"]["Europe"].keys())

In [9]:
country_name="Germany"

In [10]:
company_list = EIC_listing_data["SSO"]["Europe"][country_name]
company_list[0]["eic"]

'21X000000001160J'

In [11]:
api_string = "https://agsi.gie.eu/api?"

In [12]:
def generate_query_string(api_string, query_string_parameter_dict):
    """_summary_

    Args:
        api_string (_type_): _description_
        query_string_parameter_dict (_type_): _description_

    Returns:
        _type_: _description_
    """
    query_string = api_string
    for query_string_parameter_name in query_string_parameter_dict:
        query_string_parameter_value = query_string_parameter_dict[query_string_parameter_name]
        query_string_parameter_pair = "=".join([query_string_parameter_name, query_string_parameter_value])
        query_string = "&".join([query_string, query_string_parameter_pair])

    return query_string

In [13]:
def generate_query_string_list(EIC_listing_data):
    query_string_list = []
    
    company_list = EIC_listing_data["SSO"]["Europe"][country_name]
    query_string_parameter_dict = {}
    for company in company_list:
        query_string_parameter_dict["company"] = company["eic"]
        facility_list = company["facilities"]
        for facility in facility_list:
            query_string_parameter_dict["facility"] = facility["eic"]
            query_string_parameter_dict["country"] = facility["country"]["code"]
            query_string = generate_query_string(api_string, query_string_parameter_dict)

            query_string_list.append(query_string)
    return query_string_list

In [14]:
def request_query_as_json(query_string, api_key=keys.AGSI):
    header = {}
    header["x-key"] = keys.AGSI
    
    resp = requests.get(query_string, headers=header)
    data = resp.json()

    return data

In [15]:
extraction_keyword_list = ["gasDayStart","gasInStorage","injection","withdrawal","workingGasVolume","status","trend","full"]

In [16]:
def decompose_query_string_into_parameter_dict(query_string):
    query_string_parameter_string_list = query_string.split("&")[1:]
    query_string_parameter_dict = {}

    for query_string_parameter_string in query_string_parameter_string_list:
        query_string_parameter_name, query_string_parameter_value  = query_string_parameter_string.split("=")
        query_string_parameter_dict[query_string_parameter_name] = query_string_parameter_value;
    
    return query_string_parameter_dict
    

In [17]:
metadata_keyword_list = ["name","url","injectionCapacity","withdrawalCapacity"]

In [21]:
def curl_data_from_query(query_string, extraction_keyword_list, metadata_keyword_list):

    initial_request_data = request_query_as_json(query_string)
    number_of_pages = initial_request_data["last_page"]
    
    # Extract Metadata from first entry
    metadata_dict = {}
    initial_request_data_first_entry = initial_request_data["data"][0]

    for metadata_keyword in metadata_keyword_list:
        metadata_dict[metadata_keyword] = initial_request_data_first_entry[metadata_keyword]
        
    # Query all pages
    page_dictionary_list = []
    for page_number in tqdm(range(number_of_pages)):
        query_string_page="&".join([query_string,"page="+str(page_number)])
        page_dictionary_list.append(request_query_as_json(query_string_page))
    
    # Extract entries from the pages
    dataseries_list = []
    for page_dictionary in page_dictionary_list: 
        for datapoint in page_dictionary["data"]:
            datapoint_list = []
            for keyword in extraction_keyword_list: 
                datapoint_list.append(datapoint[keyword])
            dataseries_list.append(datapoint_list)
    
    dataseries_df = pd.DataFrame(dataseries_list, columns = extraction_keyword_list)
    dataseries_df.attrs = metadata_dict
    return dataseries_df
    

In [25]:
query_string_list = generate_query_string_list(EIC_listing_data)

In [28]:
facility_dataseries = curl_data_from_query(query_string_list[0], extraction_keyword_list, metadata_keyword_list)

 21%|██        | 30/143 [00:29<01:53,  1.00s/it]